In [1]:
import pandas as pd
from sqlalchemy import create_engine, text
from bokeh.plotting import figure, output_file, output_notebook, show, save
from bokeh.models import FuncTickFormatter, FixedTicker, NumeralTickFormatter
from bokeh.charts import Bar, Donut
import gc
from IPython.core.display import display, HTML

display(HTML("<style>.container { width:100% !important; }</style>"))

# $ echo 'cvodb2-01:5432:cvodb:cadcuws:bc5e755f2be8e8' >> ~/.pgpass && chmod 600 .pgpass
# $ echo 'cvodb1:5432:cvodb:cadcuws:bc5e755f2be8e8' >> ~/.pgpass
engine = create_engine('postgresql://cadcuws@cvodb2-01:5432/cvodb')
engine_old = create_engine('postgresql://cadcuws@cvodb1:5432/cvodb')

In [2]:
begin = pd.read_sql_query(text("SELECT date_trunc('day', job.starttime::DATE) AS starttime \
    FROM uws.job job INNER JOIN uws.jobdetail jd ON job.jobid = jd.jobid \
    WHERE job.starttime IS NOT NULL \
    AND jd.value LIKE '%Observation.observationURI AS \"Preview\"%'\
    ORDER BY 1 ASC\
    LIMIT 1"), con = engine_old).at[0, "starttime"].strftime('%Y-%m-%d')

print(begin)

end = pd.read_sql_query(text("SELECT date_trunc('day', job.starttime::DATE) AS starttime \
    FROM uws.job job INNER JOIN uws.jobdetail jd ON job.jobid = jd.jobid \
    WHERE job.starttime IS NOT NULL \
    AND jd.value LIKE '%Observation.observationURI AS \"Preview\"%'\
    ORDER BY 1 DESC\
    LIMIT 1"), con = engine).at[0, "starttime"].strftime('%Y-%m-%d')

print(end)

2012-11-26
2017-03-15


In [3]:
query = "SELECT\
    COUNT(1) AS Tot_q,\
    COUNT(CASE WHEN (jd.value LIKE '%lower(Observation.observationID) =%' \
            OR jd.value LIKE '%lower(Observation.observationID) LIKE%') THEN 1 END) AS Observation_ID,\
    COUNT(CASE WHEN jd.value LIKE '%lower(Observation.proposal_pi) LIKE%' THEN 1 END) AS PI_name,\
    COUNT(CASE WHEN jd.value LIKE '%lower(Observation.proposal_id) LIKE%' THEN 1 END) AS proposal_id,\
    COUNT(CASE WHEN jd.value LIKE '%lower(Observation.proposal_title) LIKE%' THEN 1 END) AS proposal_title,\
    COUNT(CASE WHEN jd.value LIKE '%lower(Observation.proposal_keywords) LIKE%' THEN 1 END) AS Propsal_keyword,\
    COUNT(CASE WHEN jd.value LIKE '%Plane.dataRelease >=% AND Plane.dataRelease <=%' THEN 1 END) AS Data_Rlease_Date,\
    COUNT(CASE WHEN (jd.value LIKE '%Plane.dataRelease <=%' AND jd.value NOT LIKE '%Plane.dataRelease >=%') THEN 1 END) AS Data_Rlease_Date_public,\
    COUNT(CASE WHEN jd.value LIKE '%lower(Observation.intent) = ''calibration''%' THEN 1 END) AS Calibration_only,\
    COUNT(CASE WHEN jd.value LIKE '%lower(Observation.intent) = ''science''%' THEN 1 END) AS Science_only,\
    \
    COUNT(CASE WHEN jd.value LIKE '%INTERSECTS( CIRCLE(''ICRS'',%), Plane.position_bounds ) = 1%' THEN 1 END) AS Target,\
    COUNT(CASE WHEN jd.value LIKE '%TAP_UPLOAD.search_upload as f on INTERSECTS(POINT(''ICRS'', f.ra, f.dec)%' THEN 1 END) AS target_upload,\
    COUNT(CASE WHEN jd.value LIKE '%Plane.position_sampleSize >=%AND  Plane.position_sampleSize <=%' THEN 1 END) AS pixel_scale,\
    \
    COUNT(CASE WHEN jd.value LIKE '%INTERSECTS( INTERVAL(%), Plane.time_bounds ) = 1%' THEN 1 END) AS observation_date,\
    COUNT(CASE WHEN jd.value LIKE '%Plane.time_exposure =%' THEN 1 END) AS Integration_time,\
    COUNT(CASE WHEN jd.value LIKE '%Plane.time_bounds_width =%' THEN 1 END) AS time_span,\
    \
    COUNT(CASE WHEN jd.value LIKE '%INTERSECTS( INTERVAL(%), Plane.energy_bounds ) = 1%' THEN 1 END) AS Spactral_Coverage,\
    COUNT(CASE WHEN jd.value LIKE '%Plane.energy_sampleSize >%AND  Plane.energy_sampleSize <%' THEN 1 END) AS Spactral_Sampling,\
    COUNT(CASE WHEN jd.value LIKE '%Plane.energy_resolvingPower >%' THEN 1 END) AS Resolving_power,\
    COUNT(CASE WHEN jd.value LIKE '%Plane.energy_bounds_width >%AND  Plane.energy_bounds_width <%' THEN 1 END) AS Bandpass_width,\
    COUNT(CASE WHEN jd.value LIKE '%Plane.energy_restwav >%AND  Plane.energy_restwav <%' THEN 1 END) AS rest_frame_energy,\
    \
    COUNT(CASE WHEN jd.value LIKE '%Plane.energy_emBand =%' THEN 1 END) AS band,\
    COUNT(CASE WHEN jd.value LIKE '%Observation.collection =%' THEN 1 END) AS Collection,\
    COUNT(CASE WHEN jd.value LIKE '%Observation.instrument_name =%' THEN 1 END) AS Instrument,\
    COUNT(CASE WHEN jd.value LIKE '%Plane.energy_bandpassName =%' THEN 1 END) AS Filter,\
    COUNT(CASE WHEN jd.value LIKE '%Plane.calibrationLevel =%' THEN 1 END) AS Calibration_level,\
    COUNT(CASE WHEN jd.value LIKE '%Plane.dataProductType =%' THEN 1 END) AS Data_type,\
    COUNT(CASE WHEN jd.value LIKE '%Observation.type =%' THEN 1 END) AS Observation_type\
    FROM uws.Job job INNER join uws.JobDetail jd ON job.jobid = jd.jobid\
    WHERE jd.value LIKE '%Observation.observationURI AS \"Preview\"%'\
    "
df_summ = pd.read_sql_query(text(query), con = engine).T
df_summ_old = pd.read_sql_query(text(query), con = engine_old).T
    
# print(df_summ_old)
# print(df_summ)

df_summ = df_summ.add(df_summ_old)
del df_summ_old

In [4]:
print(df_summ.sort_values(0, ascending = False))
tot = df_summ.at["tot_q", 0]
df_sum = df_summ / tot
p = figure(width = 900, title = "Advanced Search: Usage of Each Field (from %s to %s)" % (begin, end))
df_sum = df_sum.sort_values(0)
translation = {'spactral_sampling': 'Spectral Sampling', 'bandpass_width': 'Bandpass Width', 'resolving_power': 'Resolving Power', 'integration_time': 'Integration Time', 'time_span': 'Time Span', 'data_rlease_date': 'Data Release Date', 'pixel_scale': 'Pixel Scale', 'rest_frame_energy': 'Rest-frame Energy', 'spactral_coverage': 'Spectral Coverage', 'calibration_only': 'Proposal Keyword', 'propsal_keyword': 'Calibration Data Only', 'proposal_title': 'Proposal Title', 'target_upload': 'Target: File Upload', 'observation_date': 'Observation Date', 'filter': 'Observation ID', 'observation_id': 'Filter', 'pi_name': 'P.I. Name', 'proposal_id': 'Proposal ID', 'band': 'Band', 'calibration_level': 'Calibration Level', 'instrument': 'Instrument', 'observation_type': 'Observation Type', 'data_type': 'Science Data Only', 'science_only': 'Data Type', 'collection': 'Collection', 'target': 'Target', 'data_rlease_date_public': 'Public Data Release Date'}
y = [ _ for _ in range(1, len(df_sum))]
d = dict(zip(y, [translation[x] for x in df_sum.index[:-1]]))
output_notebook()
p.hbar(y = y, right = df_sum[0][:-1], height = 0.5, left = 0)
p.xaxis.axis_label = ("Percentage (Total Number of Queries %i)" % tot)
p.yaxis.axis_label = "Fields"
p.yaxis[0].ticker = FixedTicker(ticks = y)
p.yaxis[0].formatter = FuncTickFormatter(code = """dic = """ + str(d) + """
     return dic[tick]""")
p.xaxis[0].formatter = NumeralTickFormatter(format = "0.%")
show(p)
del p
gc.collect()

                              0
tot_q                    727110
instrument               367458
target                   355927
calibration_level        314529
band                     299819
data_type                298917
observation_type         297552
science_only             291969
collection                97497
filter                    15297
proposal_id               11860
pi_name                    9870
observation_id             7735
observation_date           7055
target_upload              2146
proposal_title             1182
data_rlease_date_public     992
calibration_only            725
propsal_keyword             696
spactral_coverage           649
rest_frame_energy           283
pixel_scale                 182
data_rlease_date            181
time_span                   103
integration_time             79
resolving_power              75
spactral_sampling            54
bandpass_width               40


Loading BokehJS ...

37

In [11]:
query = "SELECT date_trunc('month', job.starttime::DATE) AS starttime, remoteip, count(*)\
                        FROM uws.job job INNER JOIN uws.jobdetail jd ON job.jobid = jd.jobid\
                        WHERE jd.value LIKE '%Observation.observationURI AS \"Preview\"%' \
                        GROUP BY 1, 2\
                        ORDER BY 1, 3 DESC\
                        "
df_perwk_perip_new = pd.read_sql_query(text(query),con=engine)
df_perwk_perip_old = pd.read_sql_query(text(query),con=engine_old)

df_perwk_perip = pd.concat([df_perwk_perip_old, df_perwk_perip_new])
del df_perwk_perip_old, df_perwk_perip_new

In [17]:
num_q_perweek = df_perwk_perip.assign(starttime = df_perwk_perip.starttime.astype("str")\
                                      .apply(lambda x: x.split(" ")[0]))
num_q_perweek = num_q_perweek.groupby("starttime").sum().reset_index()
p1 = Bar(num_q_perweek, "starttime", "count", legend = False, plot_width = 900,\
        xlabel = 'Date (Every Four Month)', ylabel = "Counts", title = "Advanced Search: Numer of Queries per Month",\
         yscale = "log")

xtickloc = [_ for _ in range(1, len(num_q_perweek) + 1)]
d = dict(zip(xtickloc, num_q_perweek["starttime"]))
newxtickloc = xtickloc[0::4]
newd = {_:d[_] for _ in newxtickloc}
p1.xaxis[0].ticker = FixedTicker(ticks = xtickloc)
p1.xaxis[0].formatter = FuncTickFormatter(code = """dic = """ + str(newd) + """
    if (tick in dic) {
        return dic[tick]
    }
    else {
        return ''
    }""")
show(p1)
del num_q_perweek, p1
gc.collect()

0

In [13]:
tmp = df_perwk_perip[["count", "remoteip"]].groupby("remoteip").sum().sort_values("count")
num_q_perip_top10 = tmp.tail(9)
num_q_perip_rest = tmp[0:-9]
tmp = sum(num_q_perip_rest.values)
num_q_perip_rest = pd.DataFrame(tmp, index = ["others"], columns = ["count"])
num_q_perip = num_q_perip_rest.append(num_q_perip_top10)

del tmp, num_q_perip_rest, num_q_perip_top10

p2 = figure(width = 900, title = "Advanced Search: Number of Queries submitted by IP (from %s to %s)" % (begin, end))
y = [ _ for _ in range(len(num_q_perip))]
d = dict(zip(y, num_q_perip.index))
p2.hbar(y = y, right = num_q_perip["count"] / tot, height = 0.5, left = 0)
p2.xaxis.axis_label = ("Percentage (Total Number of Queries %i)" % tot)
p2.yaxis.axis_label = "IP"
p2.yaxis[0].ticker = FixedTicker(ticks = y)
p2.yaxis[0].formatter = FuncTickFormatter(code = """dic = """ + str(d) + """
     return dic[tick]""")
p2.xaxis[0].formatter = NumeralTickFormatter(format = "0.%")
show(p2)
del num_q_perip, p2
gc.collect()

47

In [24]:
num_q_perip_201601 = df_perwk_perip.assign(starttime = df_perwk_perip.starttime.astype("str")\
                        .apply(lambda x: x.split(" ")[0]))
num_q_perip_201601 = num_q_perip_201601[num_q_perip_201601.starttime == "2016-01-01"][["remoteip", "count"]]\
                        .sort_values("count").tail(5)
p3 = figure(width = 900, title = "Number of Queries submitted in 2016-01 by IP (top 5)")
y = [ _ for _ in range(len(num_q_perip_201601))]
d = dict(zip(y, num_q_perip_201601["remoteip"]))
p3.hbar(y = y, right = num_q_perip_201601["count"], height = 0.5, left = 0)
p3.xaxis.axis_label = "Counts"
p3.yaxis.axis_label = "IP"
p3.yaxis[0].ticker = FixedTicker(ticks = y)
p3.yaxis[0].formatter = FuncTickFormatter(code = """dic = """ + str(d) + """
     return dic[tick]""")
show(p3)
del num_q_perip_201601, p3
gc.collect()

14

In [15]:
query = "SELECT AVG(count) FROM (\
    SELECT count(*)\
    FROM uws.Job job INNER JOIN uws.JobDetail jd ON job.jobid = jd.jobid\
    WHERE jd.value LIKE '%Observation.observationURI AS \"Preview\"%'\
    GROUP BY job.starttime::DATE\
    ) AS avg_q_perday\
    "
df_QperDay_new = pd.read_sql_query(text(query), con = engine)
df_QperDay_old = pd.read_sql_query(text(query), con = engine_old)
print("avg query per day: %i" % (df_QperDay_new.at[0,"avg"] + df_QperDay_old.at[0,"avg"]))
del df_QperDay_new, df_QperDay_old
gc.collect()

avg query per day: 828


90

In [25]:
#import ipaddress as ip
query = "SELECT ( \
CASE \
WHEN ( job.remoteip ::inet BETWEEN '132.246.0.0'::inet AND '132.246.255.255'::inet) THEN \
    CASE \
    WHEN ((job.remoteip ::inet BETWEEN '132.246.194.0'::inet AND '132.246.194.255'::inet) \
            OR (job.remoteip ::inet BETWEEN '132.246.195.0'::inet AND '132.246.195.255'::inet) \
            OR (job.remoteip ::inet BETWEEN '132.246.217.0'::inet AND '132.246.217.255'::inet)) THEN 'CADC' \
    ELSE 'NRC w/o CADC' \
    END \
WHEN (job.remoteip ::inet BETWEEN '206.12.0.0'::inet AND '206.12.255.255'::inet) THEN 'Compute Canada' \
WHEN (job.remoteip ::inet BETWEEN '192.168.0.0'::inet AND '192.168.255.255'::inet) THEN 'CADC' \
ELSE 'Others' \
END \
) AS domain, job.remoteip \
FROM uws.Job job INNER JOIN uws.JobDetail jd ON job.jobid = jd.jobid \
WHERE jd.value LIKE '%Observation.observationURI AS \"Preview\"%'\
    "
df_domain_new = pd.read_sql_query(text(query), con = engine)
df_domain_old = pd.read_sql_query(text(query), con = engine_old)
df_domain = pd.concat([df_domain_old, df_domain_new])
df_domain = df_domain.assign(event = 1)
del df_domain_new, df_domain_old

In [27]:
grped_domain = df_domain.groupby("domain").count().reset_index()
print(grped_domain)
output_notebook()
p4 = Donut(grped_domain, label = 'domain', values = 'event', plot_width = 600, plot_height = 600, \
           title = "Advanced Search: Queries Percentage Submitted by Domains (from %s to %s)" % (begin, end))
show(p4)
del p4
gc.collect()

           domain  remoteip   event
0            CADC    122499  122499
1  Compute Canada    289939  289939
2    NRC w/o CADC        38      38
3          Others    314384  314384


Loading BokehJS ...

35

In [28]:
df_domain2 = df_domain.groupby(["domain", "remoteip"]).count().reset_index()
print("total unique ip in others %i" % len(df_domain2[df_domain2.domain == "Others"]))
tmp1 = df_domain2[df_domain2.domain == "Compute Canada"].sort_values("event").tail(4)
tmp2 = df_domain2[df_domain2.domain == "CADC"].sort_values("event").tail(4)
tmp3 = df_domain2[df_domain2.domain == "Others"].sort_values("event").tail(4)
tmp4 = df_domain2[df_domain2.domain == "NRC w/o CADC"].sort_values("event").tail(1)
df_domain2 = pd.concat([tmp1, tmp2, tmp3, tmp4])
df_domain2 = df_domain2.sort_values(["domain","event"])
print(df_domain2)
output_notebook()
p5 = Donut(df_domain2, label = ["domain", "remoteip"], values = "event", plot_width = 900, plot_height = 900, \
           title = "Advanced Search: Queries Percentage Submitted by Domains (from %s to %s)" % (begin, end))
show(p5)
del p5, df_domain2, tmp1, tmp2, tmp3, tmp4
gc.collect()

total unique ip in others 687
             domain        remoteip   event
38             CADC   132.246.217.5   23841
37             CADC   132.246.217.4   26850
40             CADC   132.246.217.7   30857
39             CADC   132.246.217.6   36648
42   Compute Canada    206.12.26.30  289939
52     NRC w/o CADC  132.246.86.118       9
140          Others    128.171.80.8   25899
683          Others  79.116.245.137   43688
711          Others    86.127.216.4   57886
499          Others   194.35.74.243  170081


Loading BokehJS ...

82

In [38]:
query = "\
SELECT ip, SUBSTRING(query_body, 'Observation\.collection\ \=\ ''([^'']+?)''') \
FROM ( \
    SELECT ( \
    CASE  \
        WHEN ( job.remoteip ::inet BETWEEN '132.246.0.0'::inet AND '132.246.255.255'::inet) THEN \
            CASE \
                WHEN ((job.remoteip ::inet BETWEEN '132.246.194.0'::inet AND '132.246.194.255'::inet) OR (job.remoteip ::inet BETWEEN '132.246.195.0'::inet AND '132.246.195.255'::inet) OR (job.remoteip ::inet BETWEEN '132.246.217.0'::inet AND '132.246.217.255'::inet)) THEN 'CADC'  \
                ELSE 'NRC' \
            END \
        WHEN (job.remoteip ::inet BETWEEN '206.12.0.0'::inet AND '206.12.255.255'::inet) THEN 'Compute Canada' \
        WHEN (job.remoteip ::inet BETWEEN '192.168.0.0'::inet AND '192.168.255.255'::inet) THEN 'CADC' \
        ELSE 'Others'  \
    END \
    ) AS domain, jd.value AS query_body, job.remoteip AS ip \
    FROM uws.Job job INNER JOIN uws.JobDetail jd ON job.jobid = jd.jobid  \
    WHERE jd.value LIKE '%Observation.observationURI AS \"Preview\"%'\
) \
AS Collection_value \
WHERE (domain = 'Others') \
"
df_Others_new = pd.read_sql_query(text(query), con = engine)
df_Others_old = pd.read_sql_query(text(query), con = engine_old)

df_Others = pd.concat([df_Others_old, df_Others_new])

del df_Others_new, df_Others_old

In [35]:
# #this cell generates ip2dom.json, a look-up json file for all the ips that are outside NRC/CADC and CC

# import json
# import socket
# from publicsuffix import fetch, PublicSuffixList

# psl_file = fetch()
# psl = PublicSuffixList(psl_file)
# ip = df_Others["ip"].value_counts().index  
# dom = [psl.get_public_suffix(socket.getfqdn(x)) for x in ip]        
# x = dict(zip(ip, dom))
# with open("ip2dom.json", "w") as f:
#     json.dump(x, f);
# del x, ip, dom

KeyboardInterrupt: 

In [47]:
#import geoip2.database
import json
import socket
from publicsuffix import fetch, PublicSuffixList

#reader = geoip2.database.Reader('/Users/will/GeoLite2-City.mmdb')

def ip2domain(s):
    # please see above cell to generate ip2dom.json
    with open("ip2dom.json", "r") as fin:
        ip2dom = json.load(fin);
    # update ip2dom.json if new ip shows ip
    psl_file = fetch()
    psl = PublicSuffixList(psl_file)    
    for x in s:
        if not x in ip2dom:
            ip2dom[x] = psl.get_public_suffix(socket.getfqdn(x))            
    with open("ip2dom.json", "w") as fout:
        json.dump(ip2dom, fout)    
    return ip2dom
    
ip2dom = ip2domain(df_Others["ip"])    
df_Others["host"] = df_Others["ip"].apply(lambda x: ip2dom[x])
# drop googlebot crawler and hawaii.edu
df_Others = df_Others[df_Others.host != "hawaii.edu"]
tot_others = len(df_Others)
s1 = df_Others["host"].value_counts(ascending = True) / tot_others
s1_top10 = s1.tail(15)
s1_rest = s1[0:-15]
others = sum(s1_rest.values)
s1_rest = pd.Series(data = others, index = ["others"])
s1 = s1_rest.append(s1_top10)
del s1_rest, s1_top10, others
output_notebook()
p6 = figure( width = 1000, height = 800, title = "Advanced Search: Number of Queries submitted by Non-NRC/CADC/CC/Hawaii.edu Domains (from %s to %s)" % (begin, end))
y = [ _ for _ in range(len(s1))]
d = dict(zip(y, s1.index))
p6.hbar(y = y, right = s1, height = 0.5, left = 0)
p6.xaxis.axis_label = ("Percentage (Total Number of Queries Submitted by Non-NRC/CADC/CC/Hawaii.edu Domains: %i)" % tot_others)
p6.yaxis.axis_label = "Domain"
p6.yaxis[0].ticker = FixedTicker(ticks = y)
p6.yaxis[0].formatter = FuncTickFormatter(code = """dic = """ + str(d) + """
     return dic[tick]""")
p6.xaxis[0].formatter = NumeralTickFormatter(format = "0.%")
show(p6)
#save(p6, "NumQueryByNonNRC-CADCDom.html")
del p6, s1
gc.collect()

Loading BokehJS ...

63

In [67]:
s2 = df_Others["substring"].value_counts( ascending = True, dropna = False) / tot_others
print(s2)
output_notebook()
p7 = figure(width = 900, height = 900, title = \
            'Most Selected "Collection" for Non-NRC/CADC/CC/Hawaii.edu ({0:.2f}% Selected "Collection")'.format(100 * (1 - s2[-1])))
y = [ _ for _ in range(len(s2) - 1)]
d = dict(zip(y, s2.index[:-1]))
p7.hbar(y = y, right = s2[:-1], height = 0.5, left = 0)
p7.xaxis.axis_label = ("Percentage (Total Number of Queries Submitted by Non-NRC/CADC/CC/Hawaii.edu Domains: %i)" % tot_others)
p7.yaxis.axis_label = 'Collection'
p7.yaxis[0].ticker = FixedTicker(ticks = y)
p7.yaxis[0].formatter = FuncTickFormatter(code = """dic = """ + str(d) + """
     return dic[tick]""")
p7.xaxis[0].formatter = NumeralTickFormatter(format = "0.00%")
show(p7)
del p7, s2
gc.collect()

OMM             0.000003
BLAST           0.000003
APASS           0.000007
HSTHLA          0.000014
IRIS            0.000017
UKIRT           0.000021
MOST            0.000021
VGPS            0.000035
CFHTWIRWOLF     0.000042
MACHO           0.000045
CGPS            0.000101
DAO             0.000121
CFHTTERAPIX     0.000121
GEMINI          0.000239
HST             0.000350
JCMT            0.002302
CFHTMEGAPIPE    0.002691
CFHT            0.003124
NaN             0.990742
Name: substring, dtype: float64


Loading BokehJS ...

551

In [54]:
query = "SELECT\
    COUNT(1) AS tot_q , \
    COUNT (CASE WHEN (jd.value LIKE '%lower(Observation.observationID) =%' \
            OR jd.value LIKE '%lower(Observation.observationID) LIKE%') THEN 1 END) AS Observation_ID,\
    COUNT (CASE WHEN jd.value LIKE '%lower(Observation.proposal_pi) LIKE%' THEN 1 END) AS PI_name,\
    COUNT (CASE WHEN jd.value LIKE '%lower(Observation.proposal_id) LIKE%' THEN 1 END) AS proposal_id,\
    COUNT (CASE WHEN jd.value LIKE '%lower(Observation.proposal_title) LIKE%' THEN 1 END) AS proposal_title,\
    COUNT (CASE WHEN jd.value LIKE '%lower(Observation.proposal_keywords) LIKE%' THEN 1 END) AS Propsal_keyword,\
    COUNT (CASE WHEN jd.value LIKE '%Plane.dataRelease >=% AND Plane.dataRelease <=%' THEN 1 END) AS Data_Rlease_Date,\
    COUNT (CASE WHEN (jd.value LIKE '%Plane.dataRelease <=%' AND jd.value NOT LIKE '%Plane.dataRelease >=%') THEN 1 END) AS Data_Rlease_Date_public,\
    COUNT (CASE WHEN jd.value LIKE '%lower(Observation.intent) = ''calibration''%' THEN 1 END) AS Calibration_only,\
    COUNT (CASE WHEN jd.value LIKE '%lower(Observation.intent) = ''science''%' THEN 1 END) AS Science_only,\
    \
    COUNT (CASE WHEN jd.value LIKE '%INTERSECTS( CIRCLE(''ICRS'',%), Plane.position_bounds ) = 1%' THEN 1 END) AS Target,\
    COUNT (CASE WHEN jd.value LIKE '%TAP_UPLOAD.search_upload as f on INTERSECTS(POINT(''ICRS'', f.ra, f.dec)%' THEN 1 END) AS target_upload,\
    COUNT (CASE WHEN jd.value LIKE '%Plane.position_sampleSize >=%AND  Plane.position_sampleSize <=%' THEN 1 END) AS pixel_scale,\
    \
    COUNT (CASE WHEN jd.value LIKE '%INTERSECTS( INTERVAL(%), Plane.time_bounds ) = 1%' THEN 1 END) AS observation_date,\
    COUNT (CASE WHEN jd.value LIKE '%Plane.time_exposure =%' THEN 1 END) AS Integration_time,\
    COUNT (CASE WHEN jd.value LIKE '%Plane.time_bounds_width =%' THEN 1 END) AS time_span,\
    \
    COUNT (CASE WHEN jd.value LIKE '%INTERSECTS( INTERVAL(%), Plane.energy_bounds ) = 1%' THEN 1 END) AS Spactral_Coverage,\
    COUNT (CASE WHEN jd.value LIKE '%Plane.energy_sampleSize >%AND  Plane.energy_sampleSize <%' THEN 1 END) AS Spactral_Sampling,\
    COUNT (CASE WHEN jd.value LIKE '%Plane.energy_resolvingPower >%' THEN 1 END) AS Resolving_power,\
    COUNT (CASE WHEN jd.value LIKE '%Plane.energy_bounds_width >%AND  Plane.energy_bounds_width <%' THEN 1 END) AS Bandpass_width,\
    COUNT (CASE WHEN jd.value LIKE '%Plane.energy_restwav >%AND  Plane.energy_restwav <%' THEN 1 END) AS rest_frame_energy,\
    \
    COUNT (CASE WHEN jd.value LIKE '%Plane.energy_emBand =%' THEN 1 END) AS band,\
    COUNT (CASE WHEN jd.value LIKE '%Observation.collection =%' THEN 1 END) AS Collection,\
    COUNT (CASE WHEN jd.value LIKE '%Observation.instrument_name =%' THEN 1 END) AS Instrument,\
    COUNT (CASE WHEN jd.value LIKE '%Plane.energy_bandpassName =%' THEN 1 END) AS Filter,\
    COUNT (CASE WHEN jd.value LIKE '%Plane.calibrationLevel =%' THEN 1 END) AS Calibration_level,\
    COUNT (CASE WHEN jd.value LIKE '%Plane.dataProductType =%' THEN 1 END) AS Data_type,\
    COUNT (CASE WHEN jd.value LIKE '%Observation.type =%' THEN 1 END) AS Observation_type \
    \
    FROM uws.Job job INNER JOIN uws.JobDetail jd ON job.jobid = jd.jobid \
    WHERE ( \
            jd.value LIKE '%Observation.observationURI AS \"Preview\"%' AND\
            /* NRC & CADC */ \
            (job.remoteip ::inet NOT BETWEEN '132.246.0.0'::inet AND '132.246.255.255'::inet) AND \
            /* CC */ \
            (job.remoteip ::inet NOT BETWEEN '206.12.0.0'::inet AND '206.12.255.255'::inet) AND \
            /* private ips */ \
            (job.remoteip ::inet NOT BETWEEN '192.168.0.0'::inet AND '192.168.255.255'::inet) AND\
            /* hawaii.edu ** this range might be incomplete or overkilled ** */ \
            (job.remoteip ::inet NOT BETWEEN '128.171.0.0'::inet AND '128.171.255.255'::inet) AND\
            (job.remoteip ::inet NOT BETWEEN '132.160.20.0'::inet AND '132.160.20.255'::inet) AND\
            (job.remoteip ::inet NOT BETWEEN '166.122.173.0'::inet AND '166.122.173.255'::inet) AND\
            (job.remoteip ::inet NOT BETWEEN '168.105.0.0'::inet AND '168.105.255.255'::inet) AND\
            /* googlebot.com */ \
            (job.remoteip ::inet NOT BETWEEN '66.249.0.0'::inet AND '66.249.255.255'::inet) \
    )\
    "
df_others_new2 = pd.read_sql_query(text(query), con = engine).T
df_others_old2 = pd.read_sql_query(text(query), con = engine_old).T    
df_Others2 = df_others_old2.add(df_others_new2)
del df_others_new2, df_others_old2
gc.collect()

80

In [65]:
tot2 = df_Others2.at["tot_q", 0]
#print(df_Others2.sort_values(0, ascending = False))
#print(df_Others2.astype("int64"))
df_sum2 = (df_Others2 / tot2).astype("float64")
p8 = figure(width = 900, title = "Advanced Search: Usage of Each Field, non-NRC/CADC/CC/hawaii.edu")
df_sum2 = df_sum2.sort_values(0)
translation = {'spactral_sampling': 'Spectral Sampling', 'bandpass_width': 'Bandpass Width', 'resolving_power': 'Resolving Power', 'integration_time': 'Integration Time', 'time_span': 'Time Span', 'data_rlease_date': 'Data Release Date', 'pixel_scale': 'Pixel Scale', 'rest_frame_energy': 'Rest-frame Energy', 'spactral_coverage': 'Spectral Coverage', 'calibration_only': 'Proposal Keyword', 'propsal_keyword': 'Calibration Data Only', 'proposal_title': 'Proposal Title', 'target_upload': 'Target: File Upload', 'observation_date': 'Observation Date', 'filter': 'Observation ID', 'observation_id': 'Filter', 'pi_name': 'P.I. Name', 'proposal_id': 'Proposal ID', 'band': 'Band', 'calibration_level': 'Calibration Level', 'instrument': 'Instrument', 'observation_type': 'Observation Type', 'data_type': 'Science Data Only', 'science_only': 'Data Type', 'collection': 'Collection', 'target': 'Target', 'data_rlease_date_public': 'Public Data Release Date'}
y = [ _ for _ in range(1, len(df_sum2))]
d = dict(zip(y, [translation[x] for x in df_sum2.index[:-1]]))
output_notebook()
p8.hbar(y = y, right = df_sum2[0][:-1], height = 0.5, left = 0)
p8.xaxis.axis_label = ("Percentage (Total Number of Queries %i)" % tot2)
p8.yaxis.axis_label = "Fields"
p8.yaxis[0].ticker = FixedTicker(ticks = y)
p8.yaxis[0].formatter = FuncTickFormatter(code = """dic = """ + str(d) + """
     return dic[tick]""")
p8.xaxis[0].formatter = NumeralTickFormatter(format = "0.2%")
show(p8)
del p8
gc.collect()

Loading BokehJS ...

21

In [56]:
query = "\
SELECT date_trunc('month', job.starttime::DATE) AS starttime, count(*)\
    FROM uws.Job job INNER JOIN uws.JobDetail jd ON job.jobid = jd.jobid \
    WHERE ( \
            jd.value LIKE '%Observation.observationURI AS \"Preview\"%' AND \
            /* NRC & CADC */ \
            (job.remoteip ::inet NOT BETWEEN '132.246.0.0'::inet AND '132.246.255.255'::inet) AND \
            /* CC */ \
            (job.remoteip ::inet NOT BETWEEN '206.12.0.0'::inet AND '206.12.255.255'::inet) AND \
            /* private ips */ \
            (job.remoteip ::inet NOT BETWEEN '192.168.0.0'::inet AND '192.168.255.255'::inet) AND\
            /* hawaii.edu ** this range might be incomplete or overkilled ** */ \
            (job.remoteip ::inet NOT BETWEEN '128.171.0.0'::inet AND '128.171.255.255'::inet) AND\
            (job.remoteip ::inet NOT BETWEEN '132.160.20.0'::inet AND '132.160.20.255'::inet) AND\
            (job.remoteip ::inet NOT BETWEEN '166.122.173.0'::inet AND '166.122.173.255'::inet) AND\
            (job.remoteip ::inet NOT BETWEEN '168.105.0.0'::inet AND '168.105.255.255'::inet) AND\
            /* googlebot.com */ \
            (job.remoteip ::inet NOT BETWEEN '66.249.0.0'::inet AND '66.249.255.255'::inet) \
    )\
    GROUP BY 1\
    ORDER BY 1 DESC"

df_others_perwk_new = pd.read_sql_query(text(query),con=engine)
df_others_perwk_old = pd.read_sql_query(text(query),con=engine_old)

df_others_perwk = pd.concat([df_others_perwk_old, df_others_perwk_new])
del df_others_perwk_old, df_others_perwk_new

In [61]:
num_q_perweek = df_others_perwk.assign(starttime = df_others_perwk.starttime.astype("str")\
                                      .apply(lambda x: x.split(" ")[0]))
num_q_perweek = num_q_perweek.groupby("starttime").sum().reset_index()
print(num_q_perweek)
p9 = Bar(num_q_perweek, "starttime", "count", legend = False, plot_width = 900,\
        xlabel = 'Date (Every Four Month)', ylabel = "Counts", title = "Advanced Search: Numer of Queries per Month, non-NRC/CADC/CC/hawaii.edu",\
         yscale = "log")
xtickloc = [_ for _ in range(1, len(num_q_perweek) + 1)]
d = dict(zip(xtickloc, num_q_perweek["starttime"]))
newxtickloc = xtickloc[0::4]
newd = {_:d[_] for _ in newxtickloc}
p9.xaxis[0].ticker = FixedTicker(ticks = xtickloc)
p9.xaxis[0].formatter = FuncTickFormatter(code = """dic = """ + str(newd) + """
    if (tick in dic) {
        return dic[tick]
    }
    else {
        return ''
    }""")
show(p9)
del num_q_perweek, p9
gc.collect()

     starttime   count
0   2013-09-01       6
1   2014-04-01       5
2   2014-05-01      42
3   2014-06-01      31
4   2014-07-01      86
5   2014-08-01     268
6   2014-09-01     314
7   2014-10-01      75
8   2014-11-01    9762
9   2014-12-01   47664
10  2015-01-01      80
11  2015-02-01   55434
12  2015-03-01      64
13  2015-04-01      74
14  2015-05-01      76
15  2015-06-01      11
16  2015-08-01      46
17  2015-09-01      78
18  2015-10-01      52
19  2015-11-01   57454
20  2015-12-01      52
21  2016-01-01  115521
22  2016-02-01      50
23  2016-03-01     198
24  2016-04-01     270
25  2016-05-01      61
26  2016-06-01      92
27  2016-07-01      51
28  2016-08-01      34
29  2016-09-01      53
30  2016-10-01      85
31  2016-11-01      61
32  2016-12-01      71
33  2017-01-01      43
34  2017-02-01      72
35  2017-03-01      46


3056

In [76]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))